In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

from tensorflow.keras.layers import Dense, LSTM, Input, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Bidirectional, Activation

In [92]:
train_x = []
train_y = []

for i in range(1,6000):
    tmp = pd.read_csv('./single_label_dataset/' + str(i) + '.csv.gzip', compression='gzip').to_numpy()
    train_x.append(tmp[:63])
    train_y.append(tmp[-1])

train_x = np.array(train_x)
train_y = np.array(train_y)



: 

In [ ]:
df = pd.DataFrame(train_y)

for i in range(23):
    print(df[df[i] > 0].shape[0])

In [ ]:
from tensorflow.keras.layers import Bidirectional, Activation

model = tf.keras.models.Sequential()
forward_layer = LSTM(256, dropout=0.2, recurrent_dropout=0.2)
backward_layer = LSTM(256, activation='relu', go_backwards=True, dropout=0.2, recurrent_dropout=0.2)
# model.add(Bidirectional(forward_layer, backward_layer=backward_layer, input_shape=(63,22)))
# model.add(Dense(22, activation='softmax'))
model.add(tf.keras.layers.Input(shape=(63,22)))
# model.add(Bidirectional(LSTM(1024, return_sequences=True)))
model.add(Bidirectional(LSTM(512, return_sequences=True, dropout=0.1 , recurrent_dropout=0.1)))
# model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(Bidirectional(LSTM(128, return_sequences=True, dropout=0.1, recurrent_dropout=0.1)))
model.add(Bidirectional(LSTM(32, dropout=0.1)))
model.add(Dense(22, activation='softmax'))
# model.add(Activation('softmax'))


In [407]:
model.compile(loss = tf.keras.losses.CategoricalCrossentropy(), optimizer='adam', metrics=['accuracy'])
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7)

In [408]:
with tf.device("/gpu:0"):
    model.fit(train_x,train_y, epochs=4000, callbacks=callback, validation_split=0.1)

Epoch 1/4000
169/169 [==============================] - 260s 2s/step - loss: 2.1895 - accuracy: 0.3236 - val_loss: 2.1911 - val_accuracy: 0.2883
Epoch 2/4000
169/169 [==============================] - 253s 1s/step - loss: 2.1294 - accuracy: 0.3482 - val_loss: 2.1587 - val_accuracy: 0.3383
Epoch 3/4000
169/169 [==============================] - 252s 1s/step - loss: 2.1244 - accuracy: 0.3336 - val_loss: 2.1675 - val_accuracy: 0.3033
Epoch 4/4000
169/169 [==============================] - 250s 1s/step - loss: 2.0750 - accuracy: 0.3536 - val_loss: 2.0156 - val_accuracy: 0.3983
Epoch 5/4000
169/169 [==============================] - 253s 1s/step - loss: 1.9004 - accuracy: 0.3880 - val_loss: 1.8979 - val_accuracy: 0.4150
Epoch 6/4000
169/169 [==============================] - 248s 1s/step - loss: 1.8041 - accuracy: 0.4132 - val_loss: 1.8422 - val_accuracy: 0.4117
Epoch 7/4000
169/169 [==============================] - 250s 1s/step - loss: 1.7334 - accuracy: 0.4323 - val_loss: 1.8075 - val_ac

In [219]:
model.save('bilstm0905_re.h5')

In [21]:
import functools
top3_acc = functools.partial(tf.keras.metrics.top_k_categorical_accuracy, k=3)

In [342]:
encoder_inputs = Input(shape=(63,22))
encoder_lstm = LSTM(units=64, return_state=True, dropout=0.2, recurrent_dropout=0.2)

encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

encoder_states = [state_h, state_c]

In [343]:
decoder_inputs = Input(shape=(None,22))
decoder_lstm = LSTM(units=64, return_state=True, dropout=0.2, recurrent_dropout=0.2)

decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state = encoder_states)

decoder_dense = Dense(22, activation='softmax')

decoder_outputs = decoder_dense(decoder_outputs)


In [86]:
from tensorflow.keras.regularizers import l2

In [386]:
units = 256
dropout = 0.4

encoder_inputs = Input(shape=(63,22))
encoder_lstm = LSTM(units=units, return_state=True, dropout=dropout, recurrent_dropout=dropout, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01))

encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

encoder_states = [state_h, state_c]


decoder_inputs = Input(shape=(None,22))
decoder_lstm = LSTM(units=units, return_state=True, dropout=dropout, recurrent_dropout=dropout, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01))

decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state = encoder_states)

decoder_dense = Dense(22, activation='softmax')

decoder_outputs = decoder_dense(decoder_outputs)


In [387]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10)

In [388]:
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

In [363]:
dec = np.zeros_like(train_x)

for i in range(dec.shape[0]):
    for j in range(dec.shape[1]):
        if j != 0:
            dec[i][j] = train_x[i][j-1]

In [389]:
with tf.device('/gpu:0'):
    model.fit(x=[train_x, dec], y=train_y, epochs=5000, callbacks=callback, validation_split=0.1)

Epoch 1/5000
169/169 [==============================] - 114s 658ms/step - loss: 3.0310 - accuracy: 0.3491 - val_loss: 2.1665 - val_accuracy: 0.3900
Epoch 2/5000
169/169 [==============================] - 112s 662ms/step - loss: 2.0563 - accuracy: 0.3921 - val_loss: 2.0777 - val_accuracy: 0.4000
Epoch 3/5000
169/169 [==============================] - 111s 657ms/step - loss: 2.0036 - accuracy: 0.3934 - val_loss: 2.0188 - val_accuracy: 0.4067
Epoch 4/5000
169/169 [==============================] - 112s 661ms/step - loss: 1.9704 - accuracy: 0.3916 - val_loss: 1.9819 - val_accuracy: 0.3967
Epoch 5/5000
169/169 [==============================] - 113s 667ms/step - loss: 1.9567 - accuracy: 0.3949 - val_loss: 1.9972 - val_accuracy: 0.4033
Epoch 6/5000
169/169 [==============================] - 112s 660ms/step - loss: 1.9412 - accuracy: 0.3866 - val_loss: 1.9683 - val_accuracy: 0.4133
Epoch 7/5000
169/169 [==============================] - 111s 654ms/step - loss: 1.9285 - accuracy: 0.3980 - val_

In [ ]:
model.save('ende_drp2.h5')

dilla.wav


In [2]:
# bd sn ht mt lt oh ch cr ri
checking_arr = [
    [1, 0, 0, 0, 0, 0, 0, 0, 0], #0
    [0, 1, 0, 0, 0, 0, 0, 0, 0], #1
    # [0, 0, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 0], #2
    [0, 0, 0, 0, 1, 0, 0, 0, 0], #3
    [0, 0, 0, 0, 0, 1, 0, 0, 0], #4
    [0, 0, 0, 0, 0, 0, 1, 0, 0], #5
    # [0, 0, 0, 0, 0, 0, 0, 1, 0], 
    [0, 0, 0, 0, 0, 0, 0, 0, 1], #6
    [1, 0, 0, 0, 0, 0, 1, 0, 0], #7
    [1, 0, 0, 0, 0, 1, 0, 0, 0], #8
    [1, 0, 0, 0, 0, 0, 0, 0, 1], #9
    [1, 0, 0, 0, 1, 0, 0, 0, 0], #10
    [1, 0, 0, 0, 0, 0, 0, 1, 0], #11
    [1, 1, 0, 0, 0, 0, 0, 0, 0], #12
    [0, 1, 0, 0, 0, 0, 1, 0, 0], #13
    [0, 1, 0, 0, 0, 1, 0, 0, 0], #14
    [0, 1, 0, 0, 0, 0, 0, 0, 1], #15
    [0, 1, 0, 0, 1, 0, 0, 0, 0], #16
    [0, 1, 0, 0, 0, 0, 0, 1, 0], #17
    [0, 0, 0, 1, 1, 0, 0, 0, 0], #18
    [1, 1, 0, 0, 0, 0, 1, 0, 0], #19
    [1, 1, 0, 0, 0, 1, 0, 0, 0], #20
    [1, 1, 0, 0, 0, 0, 0, 0, 1], #21
    [0, 0, 0, 0, 0, 0, 0, 0, 0] #22
]


seven_kind_arr = [
   # bd sn mt lt oh ch ri 
    [1, 0, 0, 0, 0, 0, 0], #0
    [0, 1, 0, 0, 0, 0, 0], #1
    [0, 0, 1, 0, 0, 0, 0], #2
    [0, 0, 0, 1, 0, 0, 0], #3
    [0, 0, 0, 0, 1, 0, 0], #4
    [0, 0, 0, 0, 0, 1, 0], #5
    [0, 0, 0, 0, 0, 0, 1], #6
    [1, 0, 0, 0, 0, 1, 0], #7
    [1, 0, 0, 0, 1, 0, 0], #8
    [1, 0, 0, 0, 0, 0, 1], #9
    [1, 0, 0, 1, 0, 0, 0], #10
    [1, 1, 0, 0, 0, 0, 0], #11
    [0, 1, 0, 0, 0, 1, 0], #12
    [0, 1, 0, 0, 1, 0, 0], #13
    [0, 1, 0, 0, 0, 0, 1], #14
    [0, 1, 0, 1, 0, 0, 0], #15
    [0, 1, 0, 0, 0, 0, 1], #16
    [0, 0, 1, 1, 0, 0, 0], #17
    [1, 1, 0, 0, 0, 1, 0], #18
    [1, 1, 0, 0, 1, 0, 0], #19
    [1, 1, 0, 0, 0, 0, 1], #20
    [0, 0, 0, 0, 0, 0, 0]
]




In [168]:
truth_loop = [7, 21, 5, 21, 12, 21, 5, 5, 5, 21, 7, 21, 12, 1, 5, 21, \
       7, 21, 5, 21, 12, 21, 5, 5, 5, 21, 7, 21, 12, 1, 5, 21, \
       7, 21, 5, 21, 12, 21, 5, 5, 5, 21, 7, 21, 12, 1, 5, 21, \
       7, 21, 5, 21, 12, 21, 5, 5, 5, 21, 7, 21, 12, 1, 5, 21, \
       7, 21, 5, 21, 12, 21, 5, 5, 5, 21, 7, 21, 12, 1, 5, 21, \
       7, 21, 5, 21, 12, 21, 5, 5, 5, 21, 7, 21, 12, 1, 5, 21, \
       7, 21, 5, 21, 12, 21, 5, 5, 5, 21, 7, 21, 12, 1, 5, 21, \
       7, 21, 5, 21, 12, 21, 5, 5, 5, 21, 7, 21, 12, 1, 5, 21, \
       7, 21, 5, 21, 12, 21, 5, 5, 5, 21, 7, 21, 12, 1, 5, 21]

# truth_loop = [7, 22, 5, 22, 12, 22, 5, 5, 5, 22, 7, 22, 12, 1, 5, 22, \
#               7, 22, 5, 22, 12, 22, 5, 5, 5, 22, 7, 22, 12, 1, 5, 22, \
#               7, 22, 5, 22, 12, 22, 5, 5, 5, 22, 7, 22, 12, 1, 5, 22, \
#               7, 22, 5, 22, 12, 22, 5, 5, 5, 22, 7, 22, 12, 1, 5, 22, \
#               7, 22, 5, 22, 12, 22, 5, 5, 5, 22, 7, 22, 12, 1, 5, 22, \
#               7, 22, 5, 22, 12, 22, 5, 5, 5, 22, 7, 22, 12, 1, 5, 22, \
#               7, 22, 5, 22, 12, 22, 5, 5, 5, 22, 7, 22, 12, 1, 5, 22, \
#               7, 22, 5, 22, 12, 22, 5, 5, 5, 22, 7, 22, 12, 1, 5, 22, \
#               7, 22, 5, 22, 12, 22, 5, 5, 5, 22, 7, 22, 12, 1, 5, 22]

other = [7, 5, 5, 5, 12, 21, 5, 5, 7, 5, 7, 5, 12, 5, 5, 5, \
       7, 5, 5, 5, 12, 21, 5, 5, 7, 5, 7, 5, 12, 5, 5, 5, \
       7, 5, 5, 5, 12, 21, 5, 5, 7, 5, 7, 5, 12, 5, 5, 5, \
       7, 5, 5, 5, 12, 21, 5, 5, 7, 5, 7, 5, 12, 5, 5, 5, \
       7, 5, 5, 5, 12, 21, 5, 5, 7, 5, 7, 5, 12, 5, 5, 5, \
       7, 5, 5, 5, 12, 21, 5, 5, 7, 5, 7, 5, 12, 5, 5, 5, \
       7, 5, 5, 5, 12, 21, 5, 5, 7, 5, 7, 5, 12, 5, 5, 5, \
       7, 5, 5, 5, 12, 21, 5, 5, 7, 5, 7, 5, 12, 5, 5, 5, \
       7, 5, 5, 5, 12, 21, 5, 5, 7, 5, 7, 5, 12, 5, 5, 5, \
       7, 5, 5, 5, 12, 21, 5, 5, 7, 5, 7, 5, 12, 5, 5, 5, \
       7, 5, 5, 5, 12, 21, 5, 5, 7, 5, 7, 5, 12, 5, 5, 5]

prd = [0, 21, 21, 21, 16, 21, 21, 21, 5,  21, 0 , 21, 1 , 0,  21, 21,\
       0, 21, 21, 21, 1,  21, 21, 21, 21, 21, 0 , 21, 1 , 0,  21, 21, \
       0, 21, 21, 21, 1,  21, 21, 21, 5,  21, 0 , 21, 1 , 0,  21, 21, \
       0, 21, 21, 21, 1,  21, 21, 21, 5,  21, 10, 21, 1 , 0,  21, 21, \
       0, 21, 21, 21, 12, 21, 0,  21, 21, 21, 0,  21, 1 , 0,  21, 21, \
       0, 21, 21, 21, 16, 21, 21, 21, 21, 21, 0,  21, 1 , 0,  21, 21, \
       0, 21, 21, 21, 1,  21, 21, 21, 5,  21, 0,  21, 1 , 10, 21, 21, \
       0, 21, 21, 21, 1,  21, 21, 21, 21, 21, 0,  21, 1 , 0,  21, 21, \
       0, 21, 21, 21, 1,  21, 0,  21, 7,  21, 0,  21, 14, 0,  21, 21, \
       0, 21, 21, 21, 1,  21, 2,  21, 21, 21, 0,  21, 1,  0,  21, 21, \
       0, 21, 21, 21, 1,  21, 21, 21, 21, 21, 0,  21, 1,  0,  21, 21, \
       0, 21, 21, 21, 12, 21, 21, 21, 5,  21, 0,  21, 1,  0,  21, 2]
ground_truth = [] # loop * 12

threefour = [7, 21, 5, 5, 5, 21, 12, 21, 5, 5, 5, 21,\
       7, 21, 5, 5, 5, 21, 12, 21, 5, 5, 5, 21,\
       7, 21, 5, 5, 5, 21, 12, 21, 5, 5, 5, 21,\
       7, 21, 5, 5, 5, 21, 12, 21, 5, 5, 5, 21,\
       7, 21, 5, 5, 5, 21, 12, 21, 5, 5, 5, 21,\
       7, 21, 5, 5, 5, 21, 12, 21, 5, 5, 5, 21,\
       7, 21, 5, 5, 5, 21, 12, 21, 5, 5, 5, 21,\
       7, 21, 5, 5, 5, 21, 12, 21, 5, 5, 5, 21,\
       7, 21, 5, 5, 5, 21, 12, 21, 5, 5, 5, 21,\
       7, 21, 5, 5, 5, 21, 12, 21, 5, 5, 5, 21,\
       7, 21, 5, 5, 5, 21, 12, 21, 5, 5, 5, 21,\
       7, 21, 5, 5, 5, 21, 12, 21, 5, 5, 5, 21]

asdf = [7, 21, 5, 21, 12, 21, 5, 21, 7, 21, 7, 21, 12, 21, 5, 21, \
        7, 21, 5, 21, 12, 21, 5, 21, 7, 21, 7, 21, 12, 21, 5, 21, \
        7, 21, 5, 21, 12, 21, 5, 21, 7, 21, 7, 21, 12, 21, 5, 21, \
        7, 21, 5, 21, 12, 21, 5, 21, 5, 21, 5, 21, 2, 2, 3, 21, \
        7, 21, 5, 21, 12, 21, 5, 21, 7, 21, 7, 21, 12, 21, 5, 21, \
        7, 21, 5, 21, 12, 21, 5, 21, 7, 21, 7, 21, 12, 21, 5, 21, \
       #  7, 21, 5, 21, 12, 21, 5, 21, 7, 21, 7, 21, 12, 21, 5, 21, \
       #  7, 21, 5, 21, 12, 21, 5, 21, 7, 21, 7, 21, 12, 21, 5, 21, \
              7, 21, 5, 21, 12, 21, 5, 21, 5, 21, 5, 21, 2, 2, 3, 21]

#1

asd = [5, 21, 7, 21, 5, 21, 12, 21, 5, 21,\
       7, 21, 5, 21, 12, 21, 5, 21, \
       7, 21, 5, 21, 12, 21, 5, 21, \
       7, 21, 7, 21, 12, 21, 5, 21, \
       7, 21, 5, 21, 12, 21, 5, 21, \
       7, 21, 5, 21, 12, 21, 5, 21, \
       7, 21, 5, 21, 12, 21, 5, 21, \
       7, 21, 7, 21, 2, 2]

asd = [21, 7, 21, 5, 21, 12, 21, 5, 21,\
       7, 21, 5, 21, 12, 21, 5, 21, \
       7, 21, 7, 21, 0,  0,  10, 21, \
       9, 21, 21, 21, 5, 21, 21, 21, \
       5, 21, 21, 21, 12, 21, 21, 21, \
       7, 21 ,21, 21, 12, 21, 0, 12, \
       5, 21, 21, 21, 12, 21, 21, 21, \
       7, 21, 21, 21, 5, 21]

asd = [
       7, 21, 5, 21, 12, 21, 5, 21, \
       7, 21, 7, 21, 0,  0,  10, 21, \
       9, 21, 21, 21, 5, 21, 21, 21, \
       5, 21, 21, 21, 12, 21, 21, 21, \
       7, 21 ,21, 21, 12, 21, 0, 12, \
       5, 21, 21, 21, 12, 21, 21, 21, \
       7, 21, 21, 21, 5, 21, #0, 21, \
       # 5, 21, 21, 21, 12, 21, 21, 21, \
       # 21, 2, 21, 2, 21, 2, 21]
]

# #2
# asd = [21, 7, 21, 7, 21, 12, 21, 5, 21, \
#        7, 21, 5, 21, 12, 21, 5, 21, \
#        7, 21, 7, 21, 12, 21, 5, 21, \
#        7, 21, 5, 21, 12, 21, 5, 21, \
#        7, 21, 7, 21, 12, 21, 5, 21, \
#        7, 21, 5, 21, 12, 21, 5, 21, \
#        7, 21, 7, 21, 12, 21, 5, 21, \
#        1, 1, 21, 2, 21, 3]


In [127]:
model = tf.keras.models.load_model('./chooseok/32(2).h5')

In [169]:
who2test = asd
prd_input = np.zeros((len(who2test), 22))
# prd_input = []
i = 0
for p in who2test:
    prd_input[i][int(p)] = 1
    i += 1

# for p in who2test:
#     prd_input.append(seven_kind_arr[p])

In [171]:
# for i in range(30):
asd = model.predict(x=np.expand_dims(prd_input[-31:], axis=0))[0]
print(np.argmax(asd))
print(asd[0].round(4))
print(asd[10].round(4))
print(asd[9].round(4))
print('------')
print(asd[2].round(4))
asd.round(3)

21
0.0475
1e-04
0.001
------
0.0011


array([0.047, 0.021, 0.001, 0.   , 0.001, 0.028, 0.   , 0.101, 0.001,
       0.001, 0.   , 0.001, 0.086, 0.003, 0.   , 0.   , 0.   , 0.   ,
       0.014, 0.   , 0.   , 0.694], dtype=float32)

In [ ]:
change = []
for i in range(prd_input.__len__() - 63):
    # a = model.predict(x=[np.expand_dims(prd_input[i:i+63], axis=0),np.expand_dims(testdec[i:i+63],axis=0)])
    a = model.predict(x=np.expand_dims(prd_input[i:i+63], axis=0))
    # print(a)
    change.append(np.argmax(a))
    change.append(a)
    # imsi = []
    # for i in a:
    #     imsi.append(i.round())
    # change.append(imsi)
        

In [116]:
change

[21,
 array([[5.65134594e-03, 3.94633114e-02, 3.38134123e-03, 4.42493969e-04,
         2.26508430e-03, 4.76730168e-02, 1.87292462e-04, 3.09590250e-04,
         2.24718708e-03, 1.79201961e-04, 2.69095199e-05, 4.13705653e-04,
         8.92597483e-04, 4.84599586e-05, 3.82965254e-06, 1.95931552e-05,
         7.64672222e-07, 1.33016219e-04, 1.07813554e-04, 1.39077201e-05,
         4.29085048e-05, 8.96496654e-01]], dtype=float32),
 7,
 array([[7.1322737e-04, 7.7466376e-04, 1.0791187e-04, 3.7272188e-05,
         1.7020659e-04, 2.8024239e-02, 3.6148187e-05, 9.4857550e-01,
         5.5231987e-03, 4.4115507e-03, 7.9494848e-06, 4.5447238e-04,
         5.1352084e-03, 3.8101382e-05, 6.6804097e-07, 1.9557234e-05,
         2.5317848e-07, 5.0105800e-06, 5.5752336e-03, 9.0990667e-05,
         2.4908888e-05, 2.7377796e-04]], dtype=float32),
 21,
 array([[5.5735058e-04, 1.1359632e-03, 3.2819582e-06, 2.4064342e-08,
         7.0135393e-06, 5.8192769e-03, 6.3336188e-06, 3.4540259e-05,
         3.9442151e-05

------
추석때 돌려놓고 갈 것

In [ ]:
units = 64
dropout = 0.4

model = tf.keras.models.Sequential()
forward_layer = LSTM(units=units, dropout=dropout, recurrent_dropout=dropout)
backward_layer = LSTM(units=units, activation='relu', go_backwards=True, dropout=dropout, recurrent_dropout=dropout)
model.add(Bidirectional(forward_layer, backward_layer=backward_layer, input_shape=(63,22)))
model.add(Dense(22, activation='softmax'))

model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
with tf.device('/gpu:0'):
    model.fit(x=train_x, y=train_y, epochs=5000, callbacks=callback, validation_split=0.1)

In [ ]:
model.save('bidunits64dropout0.4.h5')

In [90]:
units = 128
dropout = 0.2

model = tf.keras.models.Sequential()
forward_layer = tf.keras.layers.GRU(units=units, dropout=dropout, recurrent_dropout=dropout, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01))
backward_layer = tf.keras.layers.GRU(units=units, activation='relu', go_backwards=True, dropout=dropout, recurrent_dropout=dropout, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01))
model.add(Bidirectional(forward_layer, backward_layer=backward_layer, input_shape=(63,22)))
model.add(Dense(22, activation='softmax'))

model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

In [91]:
with tf.device('/gpu:0'):
    model.fit(x=train_x, y=train_y, epochs=5000, callbacks=callback, validation_split=0.1)

NameError: name 'train_x' is not defined

In [ ]:
model.save('bidunits128dropout0.2l2norm.h5')

In [ ]:
units = 64
dropout = 0.4

encoder_inputs = Input(shape=(63,22))
encoder_lstm = LSTM(units=units, return_state=True, dropout=dropout, recurrent_dropout=dropout)

encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

encoder_states = [state_h, state_c]


decoder_inputs = Input(shape=(None,22))
decoder_lstm = LSTM(units=units, return_state=True, dropout=dropout, recurrent_dropout=dropout)

decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state = encoder_states)

decoder_dense = Dense(22, activation='softmax')

decoder_outputs = decoder_dense(decoder_outputs)


In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10)

model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
with tf.device('/gpu:0'):
    model.fit(x=[train_x, dec], y=train_y, epochs=5000, callbacks=callback, validation_split=0.1)

In [ ]:
model.save('unit64dropout0.4.h5')

In [ ]:
units = 512
dropout = 0.4

encoder_inputs = Input(shape=(63,22))
encoder_lstm = LSTM(units=units, return_state=True, dropout=dropout, recurrent_dropout=dropout)

encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

encoder_states = [state_h, state_c]


decoder_inputs = Input(shape=(None,22))
decoder_lstm = LSTM(units=units, return_state=True, dropout=dropout, recurrent_dropout=dropout)

decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state = encoder_states)

decoder_dense = Dense(22, activation='softmax')

decoder_outputs = decoder_dense(decoder_outputs)


In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10)

model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
with tf.device('/gpu:0'):
    model.fit(x=[train_x, dec], y=train_y, epochs=5000, callbacks=callback, validation_split=0.1)

In [ ]:
model.save('unit512dropout0.4.h5')

In [ ]:
units = 256
dropout = 0.1

encoder_inputs = Input(shape=(63,22))
encoder_lstm = LSTM(units=units, return_state=True, dropout=dropout, recurrent_dropout=dropout, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01))

encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

encoder_states = [state_h, state_c]


decoder_inputs = Input(shape=(None,22))
decoder_lstm = LSTM(units=units, return_state=True, dropout=dropout, recurrent_dropout=dropout, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01))

decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state = encoder_states)

decoder_dense = Dense(22, activation='softmax')

decoder_outputs = decoder_dense(decoder_outputs)


In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10)

model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
with tf.device('/gpu:0'):
    model.fit(x=[train_x, dec], y=train_y, epochs=5000, callbacks=callback, validation_split=0.1)

In [ ]:
model.save('unit256dropout0.1l2norm.h5')